# Pytorch
Для более сложных моделей, например seq2seq, простого keras будет недостаточно. В основном, для этого используется PyTorch или Tensorflow. Здесь мы попытаемся разобрать основы работы с PyTorch. Для этого пройдемся по туториалам с официального сайта. Начнем с введения: https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#sphx-glr-beginner-blitz-tensor-tutorial-py    

## Установка
Если вам не нужна поддержка GPU или CUDA уже поставлена, то достаточно поставить с помощью команды, появляющейся после выбора соответвующией конфигурации: https://pytorch.org/get-started/locally/

Если же у вас Windows, то работает этот туториал:https://medium.com/@jjc7ru/one-stop-shop-for-all-your-windows-10-anaconda-pytorch-gpu-cuda10-setup-ad732fad67f1

С некоторыми модификациями:
- сейчас актуальная версия CUDA 10.1 - выбираем везде её
- При установке CUDA захочет Visual Studio, теоретически можно поставить и без неё, но лучше чтоб наверняка. С Visul Studio 2019 Community с установкой средств разработки для C++ все поставилось.

На Ubuntu CUDA вроде ставится из терминала, основная проблема - поставить драйвера.

Перейдем же наконец с самому PyTorch

In [1]:
import torch

/Users/16971921/.conda/envs/py38_venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Тензоры
Для начала расссмотрим операции над тензорами

In [2]:
# Пустая матрица 5 на 3
x = torch.empty(5,3)
print(x)

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.3484e-33, 1.4013e-45, 1.3484e-33],
        [1.4013e-45, 1.3616e-33, 1.4013e-45],
        [1.3615e-33, 1.4013e-45, 1.3618e-33]])


In [3]:
# Аналогично с рандомной инициализацией
x = torch.rand(5, 3)
print(x)

tensor([[0.9946, 0.6112, 0.8494],
        [0.6190, 0.7870, 0.8777],
        [0.5071, 0.8427, 0.5563],
        [0.7449, 0.4453, 0.8788],
        [0.7519, 0.9595, 0.7754]])


In [4]:
# Заполненная нулями и укзаным типом
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [5]:
# Также можно конструировать тензоры из питонячих списков
x = torch.tensor([5.5, 3])
print(x) # тут увидим, что тип приаедтся

tensor([5.5000, 3.0000])


In [6]:
# Создание тензеров, похожих на существуюшие 
x = x.new_ones(5, 3, dtype=torch.double)
print(x)

x = torch.randn_like(x, dtype=torch.float)
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[-0.0465,  0.4181, -0.3283],
        [ 1.9096,  0.0219, -1.1346],
        [ 1.0079, -0.1110,  0.5466],
        [-0.1819, -0.2293, -1.3828],
        [-0.2280, -0.6335,  0.8569]])


Как мы видим инициализировать тензоры можно множеством способов и делается это вполне интуитивно.

## Операции
С тензорами можно проводить различные операции, причем также множеством способов. 

In [7]:
# Получения размерностей
print(x.size()) # можно взаимодействовать как с кортежем

# сложение
y = torch.rand(5,3)
print(y)
z1 = x + y
z2 = torch.add(x,y)
z3 = x.add(y)
# Все эти записи идентичты
print (torch.norm(z1- z2), torch.norm(z1 - z3), torch.norm(z2 -z3))

# Также есть модификация in-place
y.add_(x)
print(y)

# Ну и срезы тоже работают
print(y[:,1])

torch.Size([5, 3])
tensor([[0.8955, 0.6902, 0.3047],
        [0.0591, 0.8776, 0.6151],
        [0.6883, 0.0229, 0.8203],
        [0.0905, 0.0913, 0.1572],
        [0.9995, 0.5613, 0.8153]])
tensor(0.) tensor(0.) tensor(0.)
tensor([[ 0.8490,  1.1083, -0.0235],
        [ 1.9687,  0.8995, -0.5195],
        [ 1.6962, -0.0881,  1.3669],
        [-0.0914, -0.1381, -1.2256],
        [ 0.7715, -0.0722,  1.6722]])
tensor([ 1.1083,  0.8995, -0.0881, -0.1381, -0.0722])


Одним из самых частых операций при работе с сетями являлется изменение размерностей тензоров. Для этого существует базовая операция `view`, а также `transpose` - для транспонирования (view для этого не подойдет), `squeeze` - для удаления фиктивных размерностей и `unsqueeze` для их добавления.

In [8]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)  # в -1 перейдут оставшиейся размерности
print(x.size(), y.size(), z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


In [9]:
x = torch.rand(2,3)
print(x)
print(x.transpose(0,1))

x = torch.tensor([1, 2, 3, 4])
print(torch.unsqueeze(x, 0))
y = torch.unsqueeze(x, 1)
print(y)
print(y.squeeze())

tensor([[0.2606, 0.2101, 0.9212],
        [0.5064, 0.5933, 0.3188]])
tensor([[0.2606, 0.5064],
        [0.2101, 0.5933],
        [0.9212, 0.3188]])
tensor([[1, 2, 3, 4]])
tensor([[1],
        [2],
        [3],
        [4]])
tensor([1, 2, 3, 4])


## Работа с NumPy
В PyTorch предусмотрены методы для создания тензоров из таковых в numpy и наоборот, из-за чего не нужно сильно запариваться с подготовкой данных, если та была сделана на numpy и использовалась, например, в keras

In [10]:
a = torch.ones(5)
print(a)
b = a.numpy()
print(b)

# также можно менять исходный тензор и тензор в numpy тоже изменится
a.add_(1)
print(a)
print(b)

tensor([1., 1., 1., 1., 1.])
[1. 1. 1. 1. 1.]
tensor([2., 2., 2., 2., 2.])
[2. 2. 2. 2. 2.]


In [11]:
# теперь в обратную сторону
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


## CUDA Тензоры
Тензоры могут быть перемещены с одного устройства на другое с помощью метода `to`

In [12]:
if torch.cuda.is_available(): # Проверяем что CUDA вообще есть
    device = torch.device("cuda")          # объявляем объект устройства
    y = torch.ones_like(x, device=device)  # создаем тензор прям на GPU
    x = x.to(device)                       # или переносим с помощью метода 'to'
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # 'to' можнт также менять тип

# own experiments

In [140]:
a = torch.rand(2,6)
print(a.size())
a

torch.Size([2, 6])


tensor([[0.9098, 0.8901, 0.2768, 0.0364, 0.2100, 0.4975],
        [0.3273, 0.1259, 0.3977, 0.8549, 0.6711, 0.2330]])

##### view - the same data as the self tensor but of a different shape

In [18]:
a.view(-1).size()

torch.Size([12])

##### squeeze - all the dimensions of input of size 1 removed.

In [33]:
a_new = a.view(3,1,2,1,1,2)
a_new.size()

torch.Size([3, 1, 2, 1, 1, 2])

In [34]:
a_new.squeeze().size()

torch.Size([3, 2, 2])

##### unsqueeze -  a dimension of size one inserted at the specified position

In [35]:
a.size()

torch.Size([2, 6])

In [39]:
a.unsqueeze(1).size()

torch.Size([2, 1, 6])

In [40]:
a.unsqueeze(1).unsqueeze(1).size()

torch.Size([2, 1, 1, 6])

In [42]:
a.unsqueeze(1).unsqueeze(1).squeeze().size()

torch.Size([2, 6])

##### transpose - the given dimensions dim0 and dim1 are swapped

In [44]:
a.size()

torch.Size([2, 6])

In [52]:
a.transpose(0,1).size()

torch.Size([6, 2])

In [58]:
b = torch.rand(2,4,7)
b.size()

torch.Size([2, 4, 7])

In [63]:
b.transpose(0,2).size()

torch.Size([7, 4, 2])

##### padding (pad (left, right, top, bottom)

In [103]:
import torch.nn.functional as F
F.pad(a, (0, 2, 0, 0), value=0).size()

torch.Size([2, 8])

##### sum

In [148]:
a + 1

tensor([[1.9098, 1.8901, 1.2768, 1.0364, 1.2100, 1.4975],
        [1.3273, 1.1259, 1.3977, 1.8549, 1.6711, 1.2330]])

In [109]:
print('size "a":', a.size())
print('size "b":', b.size())

size "a": torch.Size([2, 6])
size "b": torch.Size([2, 4, 7])


<b> The size of tensor a (6) must match the size of tensor b (7) at non-singleton dimension 2 </b>

In [137]:
a_new = F.pad(a, (0,2))
a_new = a_new.view(2,4,2)
a_new = F.pad(a_new, (0,5))
a_new.size()

torch.Size([2, 4, 7])

In [138]:
a_new + b

tensor([[[0.9552, 0.8251, 0.5287, 0.2246, 0.5642, 0.1204, 0.7499],
         [1.1331, 0.7273, 0.2868, 0.9403, 0.1954, 0.2629, 0.2350],
         [0.8802, 1.7028, 0.7381, 0.8874, 0.8019, 0.8475, 0.4830],
         [0.1206, 0.6260, 0.3379, 0.7733, 0.3185, 0.9651, 0.1322]],

        [[1.4241, 1.4767, 0.8398, 0.2521, 0.0185, 0.0623, 0.9509],
         [1.3656, 1.3942, 0.7359, 0.3367, 0.6506, 0.7893, 0.0812],
         [0.9041, 1.5118, 0.8635, 0.7528, 0.0934, 0.6783, 0.3384],
         [0.1313, 0.6252, 0.4128, 0.0687, 0.2544, 0.5716, 0.6000]]])

##### numpy

tensor -> numpy (изменение тензора изменяет numpy array) <br>
numpy -> tensor (изменение numpy array не изменяет тензор)

In [149]:
a_np = a.numpy()
a_np

array([[0.90979826, 0.89008623, 0.2767753 , 0.03636801, 0.21004218,
        0.4975348 ],
       [0.3273155 , 0.12594438, 0.39773566, 0.85494477, 0.6711241 ,
        0.23301935]], dtype=float32)

In [151]:
a.add_(1)
a_np

array([[1.9097983, 1.8900862, 1.2767754, 1.036368 , 1.2100422, 1.4975348],
       [1.3273156, 1.1259444, 1.3977356, 1.8549447, 1.6711241, 1.2330194]],
      dtype=float32)

In [157]:
g = np.array([1,3,-1,7])

a_tch = torch.from_numpy(g)
a_tch

tensor([ 1,  3, -1,  7])

In [158]:
g = g+1
a_tch

tensor([ 1,  3, -1,  7])